In [1]:
import ast

# rows_review_v1 = []
# with open("australian_user_reviews.json") as f:
#     for line in f:
#         line = line.strip()
#         if line:
#             rows_review_v1.append(ast.literal_eval(line))

rows_ui_v1 =[]
with open("australian_users_items.json") as f:
    for line in f:
        line = line.strip()
        if line:
            rows_ui_v1.append(ast.literal_eval(line))

In [2]:
# len(rows_review_v1), len(rows_ui_v1)

In [3]:
# pip install gensim

In [4]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [5]:
ui_rows = []
item_name_lookup = {}

for user in rows_ui_v1:
    uid = user["user_id"]

    for item in user["items"]:
        item_id = item["item_id"]
        item_name = item["item_name"]

        ui_rows.append({
            "user_id": uid,
            "item_id": item_id,
            "item_name": item_name,
            "playtime": item["playtime_forever"],
            "purchase": 1
        })

        item_name_lookup[item_id] = item_name

df_ui = pd.DataFrame(ui_rows)

In [6]:
# Label: never played
df_ui["never_play"] = (df_ui["playtime"] == 0).astype(int)

## ** Could probably build some EDA with df_ui later

### ex. avg games each user bought,...

In [7]:
# Build Negative Samples
all_items = set(df_ui["item_id"].unique())

neg_samples = []
random.seed(42)

for user, group in tqdm(df_ui.groupby("user_id")):
  purchased = set(group["item_id"])
  not_bought = list(all_items - purchased)

  # Sample up to 20 negative items per user
  sample_count = min(20, len(not_bought))
  sampled_items = random.sample(not_bought, sample_count)

  for item in sampled_items:
    neg_samples.append({
        "user_id": user,
        "item_id": item,
        "item_name": item_name_lookup[item],
        "playtime": 0,
        "purchase": 0,
        "never_play": 1
    })

df_neg = pd.DataFrame(neg_samples)

df_neg

100%|███████████████████████████████████████████████████████████████████████████| 70912/70912 [00:31<00:00, 2271.69it/s]


,user_id,item_id,item_name,playtime,purchase,never_play
0,--000--,104020,Storm in a Teacup,0,0,1
1,--000--,449220,A grande bagunça espacial - The big space mess,0,0,1
2,--000--,346680,Black Mesa Dedicated Server,0,0,1
3,--000--,215510,Rocketbirds: Hardboiled Chicken,0,0,1
4,--000--,406930,Epic Showdown,0,0,1
...,...,...,...,...,...,...
1418235,zzzmidmiss,277430,Halo: Spartan Assault,0,0,1
1418236,zzzmidmiss,63500,Swords and Soldiers HD,0,0,1
1418237,zzzmidmiss,296490,GemCraft - Chasing Shadows,0,0,1
1418238,zzzmidmiss,441456,Medieval Blocks - Substance Designer Tutorial ...,0,0,1


In [8]:
# Merge pos and neg datasets together
df_full = pd.concat([df_ui, df_neg], ignore_index = True)

df_full["target_buy"] = df_full["purchase"]
df_full["target_never"] = df_full["never_play"]

df_full

,user_id,item_id,item_name,playtime,purchase,never_play,target_buy,target_never
0,76561197970982479,10,Counter-Strike,6,1,0,1,0
1,76561197970982479,20,Team Fortress Classic,0,1,1,1,1
2,76561197970982479,30,Day of Defeat,7,1,0,1,0
3,76561197970982479,40,Deathmatch Classic,0,1,1,1,1
4,76561197970982479,50,Half-Life: Opposing Force,0,1,1,1,1
...,...,...,...,...,...,...,...,...
6571444,zzzmidmiss,277430,Halo: Spartan Assault,0,0,1,0,1
6571445,zzzmidmiss,63500,Swords and Soldiers HD,0,0,1,0,1
6571446,zzzmidmiss,296490,GemCraft - Chasing Shadows,0,0,1,0,1
6571447,zzzmidmiss,441456,Medieval Blocks - Substance Designer Tutorial ...,0,0,1,0,1


‼ Figure out later why only create sequences for Purchased only

In [9]:
# Build User Sequences for Item2Vec --> For Only Purchased Items

user_sequences = (
    df_ui[df_ui["purchase"] == 1]
    .groupby("user_id")["item_id"]
    .apply(list)
    .tolist()

)

In [10]:
# Train Item2Vec
from gensim.models import word2vec
word2vec.FAST_VERSION = -1  # disable C-optimized code

item2vec = Word2Vec(
    sentences=user_sequences,
    vector_size=64,
    window=10,
    min_count=1,
    workers=1,   # keep this at 1 for fewer moving parts
    sg=1,
)

# Each item_id now gets a embedding vector of length 64

Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_fl

In [11]:
print(len(item2vec.wv))                      # how many items got vectors?
print(df_ui['item_id'].nunique())
print(item2vec.wv.vector_size)              # should be 64
print(item2vec.wv.most_similar(0))          # or some item_id that exists

10978
10978
64
[('730', 0.8175432682037354), ('202970', 0.7723994851112366), ('24240', 0.7545332908630371), ('202990', 0.745722770690918), ('55230', 0.7390444874763489), ('104700', 0.7331860065460205), ('212910', 0.7208510041236877), ('42690', 0.7203114628791809), ('206210', 0.7169899940490723), ('113200', 0.7107676863670349)]


In [12]:
item_vecs = {item: item2vec.wv[item] for item in item2vec.wv.index_to_key}

In [13]:
df_full["item_vec"] = df_full["item_id"].map(item_vecs)
df_full = df_full[df_full["item_vec"].notna()]

In [ ]:
# Expand embedding columns
for i in range(64):
    df_full[f"v{i}"] = df_full["item_vec"].apply(lambda x: x[i])

In [ ]:
# Add extra features
item_pop = df_ui.groupby("item_id")["playtime"].count()
user_never_rate = df_ui.groupby("user_id")["never_play"].mean()

df_full["item_popularity"] = df_full["item_id"].map(item_pop)
df_full["user_never_rate"] = df_full["user_id"].map(user_never_rate)

# Fill NaNs (should be few)
df_full["item_popularity"] = df_full["item_popularity"].fillna(0)
df_full["user_never_rate"] = df_full["user_never_rate"].fillna(0)

In [ ]:
# Train Stage 1 Model --> Predicting if user will buy the game or not
features = [f"v{i}" for i in range(64)] + ["item_popularity", "user_never_rate"]

# QUESTION: MODEL DIDN'T USE item_name at all???

X = df_full[features].values
y = df_full["target_buy"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify = y
)

clf_buy = LogisticRegression(max_iter=2500)
clf_buy.fit(X_train, y_train)

print("\n===== Stage 1: Buy Prediction Model =====")
pred = clf_buy.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))
# print("F1:", f1_score(y_test, pred))
# print(confusion_matrix(y_test, pred))
print(f"num iter: {clf_buy.n_iter_}")

In [ ]:
# Train Stage 2 Model --> Predict if user will never play the game if buy

purchased_df = df_full[df_full["purchase"] == 1]

X2 = purchased_df[features].values
y2 = purchased_df["target_never"].values

# Split inside the purchased buyers only
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.2, random_state=42, stratify=y2
)

clf_never = LogisticRegression(max_iter=3000)
clf_never.fit(X2_train, y2_train)

print("\n===== Stage 2: NEVER PLAY MODEL =====")
pred2 = clf_never.predict(X2_test)
print("Accuracy:", accuracy_score(y2_test, pred2))
# print("F1:", f1_score(y2_test, pred2))
# print(confusion_matrix(y2_test, pred2))

In [ ]:
# Final Prediction Function

def predict_buy_and_never_play(user_id, item_id):
    """
    Returns probability that the user will:
    (1) Buy the item
    (2) Never play it after buying
    """
    if item_id not in item_vecs:
        return None  # unknown item

    vec = item_vecs[item_id]
    pop = item_pop.get(item_id, 0)
    unr = user_never_rate.get(user_id, 0)

    feats = np.array(list(vec) + [pop, unr]).reshape(1, -1)

    p_buy = clf_buy.predict_proba(feats)[0, 1]
    p_never = clf_never.predict_proba(feats)[0, 1]

    return p_buy * p_never

print("\nExample prediction:")
print(predict_buy_and_never_play(df_ui.iloc[0]['user_id'],
                                 df_ui.iloc[0]['item_id']))

In [ ]:
# test_data = df_full[["user_id", "item_id", "target_buy", "target_never"]]

# test_data["buy_and_never"] = test_data["target_buy"] * test_data["target_never"]



In [ ]:
# # Step 1 — take only 20% of test_data
# test_sample = test_data.sample(frac=0.10, random_state=42)

# # Step 2 — shuffle the sampled rows
# test_sample = test_sample.sample(frac=1, random_state=1337).reset_index(drop=True)

# test_sample

In [ ]:
# Pre-extract numpy arrays from test_sample
uids = test_data["user_id"].values
iids = test_data["item_id"].values

probs = np.empty(len(test_data), dtype=float)

for i in range(len(test_data)):
    p = predict_buy_and_never_play(uids[i], iids[i])
    probs[i] = np.nan if p is None else p

test_data["prob"] = probs
test_eval = test_data.dropna(subset=["prob"]).reset_index(drop=True)
test_eval

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

thresholds = np.linspace(0, 1, 101)   # 0.00, 0.01, ..., 1.00

best_acc = -1
best_thr_acc = None

best_f1 = -1
best_thr_f1 = None

results = []   # optional: store all results

y_true = test_eval["buy_and_never"].values
probs   = test_eval["prob"].values

for thr in thresholds:
    y_pred = (probs >= thr).astype(int)

    acc = accuracy_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred, zero_division=0)

    results.append((thr, acc, f1))

    if acc > best_acc:
        best_acc = acc
        best_thr_acc = thr

    if f1 > best_f1:
        best_f1 = f1
        best_thr_f1 = thr

print("===== BEST ACCURACY THRESHOLD =====")
print("Threshold:", best_thr_acc)
print("Accuracy:", best_acc)

print("\n===== BEST F1 THRESHOLD =====")
print("Threshold:", best_thr_f1)
print("F1:", best_f1)
